In [22]:
import os, io, zipfile, json, time
from datetime import datetime
import requests
import pandas as pd

RAW_DIR = "../data/raw"
PROC_DIR = "../data/processed"
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)

In [23]:
# 1) DOWNLOAD MONTHLY ZIPS
import os, time, requests

BASE = "https://s3.amazonaws.com/tripdata"
RAW_DIR = r"C:\Users\moein\anaconda3\citi-bike-2022-weather\data\raw"
os.makedirs(RAW_DIR, exist_ok=True)

months = [f"2022{m:02d}" for m in range(1, 13)]
# Standard pattern + July fallback with the known typo
def candidates(ym):
    if ym == "202207":
        return [
            f"JC-{ym}-citibike-tripdata.csv.zip",  # preferred (try first)
            f"JC-{ym}-citbike-tripdata.csv.zip",   # fallback (actual typo on S3)
        ]
    else:
        return [f"JC-{ym}-citibike-tripdata.csv.zip"]

def try_download(url, outpath):
    with requests.get(url, stream=True, timeout=60) as r:
        if r.status_code == 404:
            return False
        r.raise_for_status()
        with open(outpath, "wb") as f:
            for chunk in r.iter_content(1024*512):
                if chunk:
                    f.write(chunk)
    return True

downloaded, skipped, missing = [], [], []
for ym in months:
    ok = False
    for fname in candidates(ym):
        url = f"{BASE}/{fname}"
        dst = os.path.join(RAW_DIR, fname)
        if os.path.exists(dst):
            skipped.append(fname)
            ok = True
            break
        if try_download(url, dst):
            downloaded.append(fname)
            ok = True
            time.sleep(0.4)
            break
    if not ok:
        missing.append(ym)

print("Downloaded:", downloaded)
print("Skipped (already present):", skipped)
print("Missing months:", missing)
print("Files in raw:", len(os.listdir(RAW_DIR)))

Downloaded: ['JC-202201-citibike-tripdata.csv.zip', 'JC-202202-citibike-tripdata.csv.zip', 'JC-202203-citibike-tripdata.csv.zip', 'JC-202204-citibike-tripdata.csv.zip', 'JC-202205-citibike-tripdata.csv.zip', 'JC-202206-citibike-tripdata.csv.zip', 'JC-202207-citbike-tripdata.csv.zip', 'JC-202208-citibike-tripdata.csv.zip', 'JC-202209-citibike-tripdata.csv.zip', 'JC-202210-citibike-tripdata.csv.zip', 'JC-202211-citibike-tripdata.csv.zip', 'JC-202212-citibike-tripdata.csv.zip']
Skipped (already present): []
Missing months: []
Files in raw: 13


In [24]:
# 2) CONCATENATE ALL INTO ONE DATAFRAME (no extraction)
import os, zipfile, io
import pandas as pd

RAW_DIR = r"C:\Users\moein\anaconda3\citi-bike-2022-weather\data\raw"
PROC_DIR = r"C:\Users\moein\anaconda3\citi-bike-2022-weather\data\processed"
os.makedirs(PROC_DIR, exist_ok=True)

# Helper to read a single ZIP -> DataFrame (robust to minor column differences)
def read_zip_csv(zip_path):
    with zipfile.ZipFile(zip_path) as z:
        # Each zip has one CSV; grab it
        names = [n for n in z.namelist() if n.lower().endswith(".csv")]
        if not names:
            return pd.DataFrame()
        with z.open(names[0]) as f:
            # Parse dates if present; ignore errors to avoid crashes
            df = pd.read_csv(
                io.TextIOWrapper(f, encoding="utf-8"),
                low_memory=False
            )
    return df

# Load and union columns (outer concat)
dfs = []
for fname in sorted(os.listdir(RAW_DIR)):
    if fname.lower().endswith(".zip") and fname.startswith("JC-2022"):
        path = os.path.join(RAW_DIR, fname)
        df = read_zip_csv(path)
        if not df.empty:
            dfs.append(df)

if not dfs:
    raise RuntimeError("No CSVs loaded. Check your raw directory and filenames.")

# Outer concat to handle column mismatches across months
all_df = pd.concat(dfs, axis=0, ignore_index=True, sort=False)

# 3) LIGHT NORMALIZATION (optional but helpful)
# Standardize likely datetime columns if they exist
for col in ["started_at", "ended_at", "starttime", "stoptime", "start_time", "end_time"]:
    if col in all_df.columns:
        all_df[col] = pd.to_datetime(all_df[col], errors="coerce")

# Standardize member type column names (varies by year/version)
membership_cols = [c for c in all_df.columns if "member" in c.lower() or "usertype" in c.lower()]
if membership_cols:
    # Pick the first as canonical
    canon = membership_cols[0]
    all_df.rename(columns={canon: "member_type"}, inplace=True)
    # Optionally drop others to avoid duplicates
    for c in membership_cols[1:]:
        if c in all_df.columns:
            all_df.drop(columns=c, inplace=True)

# 4) SAVE
out_csv = os.path.join(PROC_DIR, "citibike_2022_all.csv")
out_parquet = os.path.join(PROC_DIR, "citibike_2022_all.parquet")
all_df.to_csv(out_csv, index=False)
try:
    all_df.to_parquet(out_parquet, index=False)  # requires pyarrow or fastparquet
except Exception as e:
    print("Parquet save skipped (install pyarrow or fastparquet).", e)

print("Rows:", len(all_df), "Cols:", len(all_df.columns))
print("Saved:", out_csv, "and (if supported) parquet.")

Rows: 895485 Cols: 13
Saved: C:\Users\moein\anaconda3\citi-bike-2022-weather\data\processed\citibike_2022_all.csv and (if supported) parquet.


In [25]:
# 5) QUICK SANITY CHECKS
import pandas as pd
all_df = pd.read_csv(r"C:\Users\moein\anaconda3\citi-bike-2022-weather\data\processed\citibike_2022_all.csv")

# Date coverage based on common columns if present
date_cols = [c for c in ["started_at", "starttime", "start_time"] if c in all_df.columns]
if date_cols:
    dcol = date_cols[0]
    all_df[dcol] = pd.to_datetime(all_df[dcol], errors="coerce")
    print("Date range:", all_df[dcol].min(), "->", all_df[dcol].max())

# Basic stats
for c in ["ride_id", "tripduration", "duration", "rideable_type"]:
    if c in all_df.columns:
        print(c, "non-null:", all_df[c].notna().sum())

Date range: 2022-01-01 00:10:20 -> 2022-12-31 23:58:26
ride_id non-null: 895485
rideable_type non-null: 895485


### Combining Monthly Citi Bike Data

The raw data for 2022 is provided as 12 separate monthly ZIP files, each containing a single CSV.  
Since the schema can vary slightly across months (e.g., different column names or additional fields),  
I used **`pd.concat(..., sort=False)` with `ignore_index=True`** to create one unified DataFrame.  

This approach is effectively an **outer join on columns** across all months:  
- If a column exists in every month, the values align correctly.  
- If a column is missing from some months, those entries are automatically filled with `NaN`.  
- This ensures no data is lost and all available fields are preserved for downstream analysis.  

After concatenation, I applied some light normalization (e.g., parsing datetime fields and standardizing membership column names)  
so that the combined dataset is consistent and ready for merging later with external weather data.

In [26]:
# Pull LaGuardia daily weather (2022) via NOAA API

import os, json, requests
from datetime import datetime

TOKEN = os.getenv("NOAA_TOKEN") or "pXfcUANbqHxxlaXzZjnEMAqEiQukZWHh"

BASE = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
params_tavg = {
    "datasetid":"GHCND",
    "datatypeid":"TAVG",
    "stationid":"GHCND:USW00014732",
    "startdate":"2022-01-01",
    "enddate":"2022-12-31",
    "limit":1000
}
headers = {"token": TOKEN}

r = requests.get(BASE, params=params_tavg, headers=headers)
r.raise_for_status()
d = r.json()
tavg_rows = d.get("results", [])

# If TAVG sparse, pull TMIN+TMAX and compute
if not tavg_rows or len(tavg_rows) < 300:
    params_tmin = params_tavg.copy(); params_tmin["datatypeid"]="TMIN"
    params_tmax = params_tavg.copy(); params_tmax["datatypeid"]="TMAX"
    rmin = requests.get(BASE, params=params_tmin, headers=headers); rmin.raise_for_status()
    rmax = requests.get(BASE, params=params_tmax, headers=headers); rmax.raise_for_status()
    tmin = {(it["date"][:10]): it["value"] for it in rmin.json().get("results", [])}
    tmax = {(it["date"][:10]): it["value"] for it in rmax.json().get("results", [])}
    keys = sorted(set(tmin) | set(tmax))
    weather = pd.DataFrame({
        "date": pd.to_datetime(keys),
        # NOAA stores temps in tenths of °C
        "avg_temp_c": [((tmin.get(k) + tmax.get(k))/2)/10.0 if (tmin.get(k) is not None and tmax.get(k) is not None) else None for k in keys]
    })
else:
    # Build from TAVG directly
    dates = [it["date"][:10] for it in tavg_rows]
    vals  = [it["value"]/10.0 for it in tavg_rows]  # tenths °C → °C
    weather = pd.DataFrame({
        "date": pd.to_datetime(dates),
        "avg_temp_c": vals
    })

weather = weather.sort_values("date").dropna(subset=["avg_temp_c"])
weather.to_csv(os.path.join(PROC_DIR, "lga_weather_2022.csv"), index=False)
weather.head(), weather.shape

(        date  avg_temp_c
 0 2022-01-01        11.6
 1 2022-01-02        11.4
 2 2022-01-03         1.4
 3 2022-01-04        -2.7
 4 2022-01-05         3.2,
 (365, 2))

In [27]:
# Merge daily trips with weather & export
import os
import pandas as pd
from pathlib import Path

PROC_DIR = r"C:\Users\moein\anaconda3\citi-bike-2022-weather\data\processed"
cb_path = os.path.join(PROC_DIR, "citibike_2022_all.csv")
wx_path = os.path.join(PROC_DIR, "lga_weather_2022.csv")

cb = pd.read_csv(cb_path, low_memory=False)
wx = pd.read_csv(wx_path, parse_dates=["date"])
wx["date"] = wx["date"].dt.date

# 1) pick a start datetime column and derive a date
start_candidates = ["started_at", "starttime", "start_time"]
start_col = next((c for c in start_candidates if c in cb.columns), None)
if start_col is None:
    raise ValueError("No start datetime column found in CitiBike data.")
cb[start_col] = pd.to_datetime(cb[start_col], errors="coerce")
cb["date"] = cb[start_col].dt.date

# 2) unify duration if available (seconds → minutes)
if "tripduration" in cb.columns:
    cb["duration_min"] = cb["tripduration"] / 60.0
elif "duration" in cb.columns:
    cb["duration_min"] = cb["duration"] / 60.0

# 3) compute daily metrics (feel free to add more later)
def member_ratio(series):
    if series.isna().all():
        return None
    s = series.astype(str).str.lower()
    # treat 'member' or 'subscriber' as members; adjust if your schema differs
    return ((s.str.contains("member") | s.str.contains("subscriber")).sum()) / len(s)

agg_dict = {
    "date": "size",  # count rides
}
daily = cb.groupby("date", as_index=False).agg(rides=("date", "size"))

if "duration_min" in cb.columns:
    daily = cb.groupby("date", as_index=False).agg(
        rides=("date", "size"),
        avg_duration_min=("duration_min", "mean")
    )

if "member_type" in cb.columns:
    tmp = cb.groupby("date")["member_type"].apply(member_ratio).reset_index(name="member_share")
    daily = daily.merge(tmp, on="date", how="left")

# 4) merge with weather on date
merged = daily.merge(wx, on="date", how="left")

# 5) save outputs
out_daily = os.path.join(PROC_DIR, "citibike_2022_daily_with_weather.csv")
merged.to_csv(out_daily, index=False)

print("Saved:", out_daily)
print(merged.head())
print("Shape:", merged.shape)

Saved: C:\Users\moein\anaconda3\citi-bike-2022-weather\data\processed\citibike_2022_daily_with_weather.csv
         date  rides  member_share  avg_temp_c
0  2022-01-01    592      0.543919        11.6
1  2022-01-02   1248      0.584936        11.4
2  2022-01-03    832      0.772837         1.4
3  2022-01-04    934      0.776231        -2.7
4  2022-01-05    914      0.750547         3.2
Shape: (365, 4)


In [28]:
# Trip-level merge (very large; only if we really need per-ride temps)

import os, pandas as pd

PROC_DIR = r"C:\Users\moein\anaconda3\citi-bike-2022-weather\data\processed"
cb = pd.read_csv(os.path.join(PROC_DIR, "citibike_2022_all.csv"), low_memory=False)
wx = pd.read_csv(os.path.join(PROC_DIR, "lga_weather_2022.csv"), parse_dates=["date"])
wx["date"] = wx["date"].dt.date

start_candidates = ["started_at", "starttime", "start_time"]
start_col = next((c for c in start_candidates if c in cb.columns), None)
cb[start_col] = pd.to_datetime(cb[start_col], errors="coerce")
cb["date"] = cb[start_col].dt.date

cb_merged = cb.merge(wx, on="date", how="left")
out_trips = os.path.join(PROC_DIR, "citibike_2022_trips_with_weather.csv")
cb_merged.to_csv(out_trips, index=False)
print("Saved:", out_trips, "Rows:", len(cb_merged))

Saved: C:\Users\moein\anaconda3\citi-bike-2022-weather\data\processed\citibike_2022_trips_with_weather.csv Rows: 895485
